In [1]:
import torch
import nibabel as nib
import numpy as np
import copy
import h5py
import os
from torch.autograd import Variable
from utility import save_numpy_to_h5_dataset, load_h5_dataset, single_frame_to_numpy
from matplotlib import pyplot as plt
from models import VAE
import gzip
import csv

plt.ion()   # interactive mode

The script below download ABIDE images one by one and encode it to the latent space via the variational encoder.
The whole dataset is too big for me to download as a whole, hence such a solution.

# Initializing pretrained Variational Autoencoder

In [2]:
net = VAE()
net.cuda()
net.load_state_dict(torch.load("VAE_80_epochs.pt")) #loading weights

# Loading data

In [3]:
import sys
if sys.version_info[0] == 3:
    from urllib.request import urlopen
else:
    from urllib import urlopen
try:
    from StringIO import StringIO
except ImportError:
    from io import BytesIO as StringIO

In [4]:
with open('images/abide_file_names.csv', 'r') as f:
    reader = csv.reader(f)
    file_names = list(reader)

In [5]:
for i in range(len(file_names)):
    try:
        url = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/func_minimal/' + file_names[i][0] + '_func_minimal.nii.gz'
        outFilePath = file_names[i][0] + '_func_minimal.nii'
        response = urlopen(url)
        compressedFile = StringIO(response.read())
        decompressedFile = gzip.GzipFile(fileobj=compressedFile)

        with open('downloaded/' + outFilePath, 'wb') as outfile:
            outfile.write(decompressedFile.read())
            
        img = nib.load('downloaded/' + outFilePath)    
        frames = img.dataobj.shape[3] #number of "frames" for a given example    
            
        Xs = []
        for j in range(frames):
            img_numpy = single_frame_to_numpy(img,j)
            img_torch = torch.from_numpy(img_numpy) #convert to torch tensor
            encoded = net.encode(Variable(img_torch.cuda())) #generate latent code (60 vbles) through the variational encoder 
            encoded_numpy =  torch.cat((encoded[0].data[0],encoded[1].data[0])).cpu().numpy()
            encoded_numpy = np.expand_dims(encoded_numpy, axis=0)
            Xs.append(encoded_numpy)

        Xa = np.vstack(Xs)
        save_numpy_to_h5_dataset('encoded_images/' + file_names[i][0], Xa)
        os.remove('downloaded/' + outFilePath) #delete nii file            
            
    except:
        outFilePath = file_names[i][0] + '_func_minimal.nii'
        print(outFilePath + ' cannot be obtained.')

no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.
no_filename_func_minimal.nii cannot be obtained.


# Some tests

In [ ]:
outFilePath = 'Pitt_0050010' + '_func_minimal.nii'

In [ ]:
img = nib.load('images/' + outFilePath)

In [ ]:
frames = img.dataobj.shape[3] #number of "frames" for a given example

In [ ]:
Xs = []
for j in range(frames):
          
    img_numpy = single_frame_to_numpy(img,j)
    img_torch = torch.from_numpy(img_numpy) #convert to torch tensor
    encoded = net.encode(Variable(img_torch.cuda())) #generate latent code (60 vbles) through the variational encoder 
    encoded_numpy =  torch.cat((encoded[0].data[0],encoded[1].data[0])).cpu().numpy()
    encoded_numpy = np.expand_dims(encoded_numpy, axis=0)
    Xs.append(encoded_numpy)
    print (i)

In [ ]:
print (Xs[181])

In [ ]:
Xa = np.vstack(Xs)
save_large_dataset('encoded_images/testowy_plik', Xa)

In [ ]:
os.remove('encoded_images/testowy_plik.h5')